In [ ]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install -c conda-forge --yes --prefix {sys.prefix} mlflow scikit-learn matplotlib seaborn pandas boto3 atom-ml[full] optuna

## Desenvolvimento

In [1]:
#Importando as bibliotecas necessárias
import sys
import os
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse

In [16]:
os.environ['GIT_PYTHON_REFRESH']="quiet"

In [2]:
#Manipulação de dados
import pandas as pd

In [4]:
# Pré-Processamento
from sklearn.preprocessing import StandardScaler

In [5]:
# Criação do modelo
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

In [6]:
#Métricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [7]:
#Ignorar avisos de atualização, etc
import warnings
warnings.filterwarnings("ignore")

In [8]:
#Gráficos
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [10]:
experiment = 'poc'
# mlflow.set_tracking_uri(f"http://{os.getenv('MLFLOW_TRACKING_URI')}:5000")
if mlflow.get_experiment_by_name(experiment) is None:
    mlflow.create_experiment(experiment, artifact_location='s3://mlflow')
mlflow.set_experiment(experiment_name=experiment)

<Experiment: artifact_location='s3://mlflow', creation_time=1691430246843, experiment_id='1', last_update_time=1691430246843, lifecycle_stage='active', name='poc', tags={}>

In [11]:
tags = {"Projeto": "Tutorial MLflow", "team": "Data Science", "dataset": "Wine"}

In [12]:
def metricas(y_test, y_predict):
    acuracia = accuracy_score(y_test, y_predict)
    precision = precision_score(y_test, y_predict,average='weighted')
    recall = recall_score(y_test, y_predict,average='weighted')
    f1 = f1_score(y_test, y_predict,average='weighted')
    return acuracia, precision, recall, f1

def matriz_confusao(y_test, y_predict):
    matriz_conf = confusion_matrix(y_test.values.ravel(), y_predict)
    fig = plt.figure()
    ax = plt.subplot()
    sns.heatmap(matriz_conf, annot=True, cmap='Blues', ax=ax);

    ax.set_xlabel('Valor Predito');
    ax.set_ylabel('Valor Real'); 
    ax.set_title('Matriz de Confusão'); 
    ax.xaxis.set_ticklabels(['Classe 1', 'Classe 2', 'Classe 3']);
    ax.yaxis.set_ticklabels(['Classe 1', 'Classe 2', 'Classe 3']);
    plt.close()
    return fig

In [13]:
df = pd.read_csv('data/wine.data', sep = ',',names=range(1, 15, 1))

train, test = train_test_split(df, random_state=42)
x_train = train.drop(columns=[1])
x_test = test.drop(columns=[1])
y_train = train[[1]]
y_test = test[[1]]

In [17]:
with mlflow.start_run(run_name='KNeighborsClassifier'):
    #Registro das tags
    mlflow.set_tags(tags)

    #Criação do modelo
    n_vizinhos = 3
    scaler = StandardScaler()
    knn = KNeighborsClassifier(n_neighbors=n_vizinhos)
    pipe = Pipeline([('scaler', StandardScaler()), ('knn', knn)])
    pipe.fit(x_train, y_train)
    #Predição dos valores de testes
    y_pred_knn = pipe.predict(x_test)

    #Métricas
    acuracia, precision, recall, f1 = metricas(y_test, y_pred_knn)
    print("Número de Vizinhos: {}\nAcurácia: {}\nPrecision: {}\nRecall: {}\nF1-Score: {}".
         format(n_vizinhos, acuracia, precision, recall, f1))
    #Matriz de confusão
    matriz_conf = matriz_confusao(y_test, y_pred_knn)
    temp_name = "confusion-matrix.png"
    matriz_conf.savefig(temp_name)
    mlflow.log_artifact(temp_name, "confusion-matrix-plots")
    try:
        os.remove(temp_name)
    except FileNotFoundError as e:
        print(f"{temp_name} file is not found")

    #Registro dos parâmetros e das métricas
    mlflow.log_param("n_neighbors", n_vizinhos)
    mlflow.log_metric("Acuracia", acuracia)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1-Score", f1)

    #Registro do modelo
    mlflow.sklearn.log_model(pipe, "model")
    mlflow.log_artifact(local_path='./main.ipynb', artifact_path='code')


2023/08/07 17:54:53 WARNING mlflow.tracking.context.registry: Encountered unexpected error during resolving tags: 'getpwuid(): uid not found: 1001'


Número de Vizinhos: 3
Acurácia: 0.9555555555555556
Precision: 0.9586538461538462
Recall: 0.9555555555555556
F1-Score: 0.9550512333965844
